# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [3]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [6]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [7]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [8]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [9]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


In [10]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,0,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,1,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,2,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,3,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,4,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


## Матрица рейтингов

In [11]:
from scipy.sparse import coo_matrix, csr_matrix

In [12]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [14]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [15]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [26]:
user_1 = R[0]
user_1.todense()

(1, 10109)

In [94]:
df_rates.shape

(855598, 9)

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [31]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [32]:
user_1_rated = (user_1 != 0)
user_1_rated.data.shape

(55,)

In [33]:
user_2_rated = (user_2 != 0)
user_2_rated.data.shape

(468,)

Можем их "индексировать"

In [34]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [35]:
user_1.nnz

55

In [36]:
user_2.nnz

468

Можем умножать 2 разреженных вектора поэлементно:

In [49]:
(user_1).multiply(user_2).data

array([22.5 , 14.  , 22.5 , 16.  ,  2.25, 18.  , 18.  , 11.25, 12.  ,
       22.5 , 14.  , 15.75, 12.  , 16.  ])

и скалярно

In [41]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [43]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

In [44]:
user_1.todense()

matrix([[0., 0., 1., ..., 0., 0., 0.]])

In [36]:
idx = (user_1.toarray() != 0) & (user_2.toarray() != 0)

count = 0

for i in idx[0]:
    if i:
        count += 1
        
count

14

In [51]:
user_rated = (user_1.todense() != 0) & (user_2.todense() != 0)

In [54]:
user_rated.data.shape

(1, 10109)

In [62]:
u = user_1[user_rated]

In [63]:
v = user_2[user_rated]

In [76]:
v.dot(v.T)[0, 0]

244.5

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


In [122]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_pair_users(u, v):
    
    if (u).multiply(v).nnz <= 2:
        return 0.0
    
    user_rated = (u.todense() != 0) & (v.todense() != 0)
    
    u = u[user_rated]
    v = v[user_rated]
    
    return u.dot(v.T)[0, 0] / ((u.dot(u.T)[0, 0] ** 0.5) *  (v.dot(v.T)[0, 0] ** 0.5))

In [123]:
cosine_similarity_pair_users(R[0], R[2])

0.0

### _Решение_

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [154]:
def get_top5(ind, M, n_neighbours):
    
    users_dict = {}
    
    for i in range(M.shape[0]):
        if i != ind:
            users_dict[i] = cosine_similarity_pair_users(M[ind], M[i])
            
    top_neighbours = sorted(users_dict, key=users_dict.get, reverse=True)[: n_neighbours]
    
    rates_dict = {}
    
    for i in range(R.toarray().shape[1]):
        if R[ind].toarray()[0][i] == 0.0:
            num_sum = 0.0
            den_sum = 0.0
            
            for j in top_neighbours:
                num_sum += users_dict[j] * R[j].toarray()[0][i]
                den_sum += users_dict[j]
            
            rates_dict[i] = num_sum / den_sum
            
    return sorted(rates_dict, key=rates_dict.get, reverse=True)[: 5]

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

In [155]:
top5 = get_top5(20, R, 30)

## Генерация ответа

In [156]:
', '.join(str(i) for i in top5)

'2614, 306, 343, 5573, 6720'